In this notebook, I am going to generate a list of companies called Dividend Aristocrats. There are some requirements for companies to be selected in the group of Dividend Aristocrats, I will make the process simple by selecting companies in S&P 500 which have paid out dividends at least 25 years. I can add more requirements on top of the threshold of 25 years.

In [35]:
import json
import pandas as pd
import yfinance as yf
import datetime as dt
from pandas_datareader.data import DataReader

## Load Annual Dividend Data from JSON

In [36]:
with open('data/historical_div_sp500.json', 'r') as fp:
    div_data = json.load(fp)

In [37]:
div_data

{'MMM': [{'1970': 0.082,
   '1971': 0.115752,
   '1972': 0.120376,
   '1973': 0.13125,
   '1974': 0.1565,
   '1975': 0.16875,
   '1976': 0.181,
   '1977': 0.2125,
   '1978': 0.25,
   '1979': 0.3,
   '1980': 0.35,
   '1981': 0.375,
   '1982': 0.4,
   '1983': 0.4125,
   '1984': 0.425,
   '1985': 0.4375,
   '1986': 0.45,
   '1987': 0.465,
   '1988': 0.53,
   '1989': 0.65,
   '1990': 0.73,
   '1991': 0.78,
   '1992': 0.8,
   '1993': 0.83,
   '1994': 0.88,
   '1995': 0.94,
   '1996': 2.16,
   '1997': 1.06,
   '1998': 1.1,
   '1999': 1.12,
   '2000': 1.16,
   '2001': 1.2,
   '2002': 1.24,
   '2003': 1.32,
   '2004': 1.44,
   '2005': 1.68,
   '2006': 1.84,
   '2007': 1.92,
   '2008': 2.0,
   '2009': 2.04,
   '2010': 2.1,
   '2011': 2.2,
   '2012': 2.36,
   '2013': 2.54,
   '2014': 3.42,
   '2015': 4.1,
   '2016': 4.44,
   '2017': 4.7,
   '2018': 5.44,
   '2019': 5.76,
   '2020': 5.88},
  {'consecutive_yrs': 51,
   '5yrsGrowthAvg': 9.623657807321878,
   '10yrsGrowthAvg': 10.45368575314163,
   

In [38]:
df_convert = {
    'Symbol': [],
    'Div Yield': [],
    '5Yrs DivGrowth Avg': [],
    '10Yrs DivGrowth Avg': [],
    '25Yrs DivGrowth Avg': [],
    'Consecutive Years of Dividend Pay': []
}

In [39]:
startDate = dt.datetime.today() - dt.timedelta(days=1)
endDate = dt.datetime.today()
last_year = dt.datetime.today().year - 1

for symbol in div_data.keys():
    df_convert['Symbol'].append(symbol)
    recent_close = DataReader(symbol, 'yahoo', startDate,
                              endDate)['Adj Close'][-1]
    df_convert['Div Yield'].append(100 * div_data[symbol][0][str(last_year)] /
                                   recent_close)
    df_convert['5Yrs DivGrowth Avg'].append(
        div_data[symbol][1]['5yrsGrowthAvg'])
    df_convert['10Yrs DivGrowth Avg'].append(
        div_data[symbol][1]['10yrsGrowthAvg'])
    df_convert['25Yrs DivGrowth Avg'].append(
        div_data[symbol][1]['25yrsGrowthAvg'])
    df_convert['Consecutive Years of Dividend Pay'].append(
    div_data[symbol][1]['consecutive_yrs'])

In [49]:
df = pd.DataFrame(df_convert)
df.set_index('Symbol', inplace=True)
df = df[(df['5Yrs DivGrowth Avg'] > 0) & (df['10Yrs DivGrowth Avg'] > 0) & (df['25Yrs DivGrowth Avg'] > 0)]
df = df.sort_values(by=['Div Yield', '5Yrs DivGrowth Avg'], ascending=False)

In [50]:
df

,Div Yield,5Yrs DivGrowth Avg,10Yrs DivGrowth Avg,25Yrs DivGrowth Avg,Consecutive Years of Dividend Pay
Symbol,,,,,
MO,6.476190,9.330357,9.030297,34.399845,59
XOM,6.030151,4.338114,7.055252,6.280630,59
IBM,4.773427,7.493477,10.729010,13.756909,59
ED,4.110693,3.289112,2.395247,1.653990,59
PFE,3.978273,6.531631,6.156811,9.929631,49
IP,3.723462,6.151018,22.483728,7.616929,59
BMY,3.581483,10.134840,8.176134,7.380885,49
AEP,3.341570,5.758080,5.131954,1.009352,51
PEG,3.305228,4.793764,3.586396,2.348500,41


In [ ]:
# import matplotlib.pyplot as plt

# x_value = [i for i in companies['MMM'][0].keys()]
# y_value = [i for i in companies['MMM'][0].values()]

# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.bar(x_value,y_value)
# # plt.plot(x_value,y_value)
# plt.title('Dividen')
# plt.xlabel('Year')
# plt.ylabel('Amount of dividen')
# plt.show()